In [1]:
try:
    spark.stop()
    print("Spark stopped")
except:
    print("NO SPARK UP")

NO SPARK UP


# Conexão SQL

In [2]:
import postgres_conn as sqlconn
import os

In [5]:
conn = sqlconn.create_server_connection('db_enem','localhost', os.getenv('USER_DB'), os.getenv('PASSWORD_DB'), '5432') # Conexão do SQL
cursor = conn.cursor() # Cursor para SQL

# Adição de dados na Tabela SQL

![](/Users/mikeiasoliveira/Documents/Projetos/análise_dados_enem/analise-dados-enem/src/sql/modelagem/modelagem_db_image.png)

## Iniciando conexão e carregamento de dados

In [6]:
import findspark
from pyspark.sql import SparkSession

findspark.init()

spark =  SparkSession.builder\
            .master('local')\
            .appName('dados_enem')\
            .config("spark.executor.extraClassPath", os.path.abspath('./'))\
            .getOrCreate()

22/11/01 14:19:16 WARN Utils: Your hostname, Mac-Mikeias.local resolves to a loopback address: 127.0.0.1; using 192.168.0.138 instead (on interface en2)
22/11/01 14:19:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 14:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df  =  spark.read.csv("/Users/mikeiasoliveira/Downloads/02 - Outros/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", 
                    header=True, 
                    inferSchema=True, 
                    sep=';')

Criando conexão com banco de dados

In [8]:
url = f"jdbc:postgresql://localhost:5432/db_enem"
user = os.getenv('USER_DB')
password = os.getenv('PASSWORD_DB')

---
## UF

Selecionando valores únicos e dropando NA

In [13]:
df_UF = df.select('CO_UF_ESC', 'SG_UF_ESC').distinct() # Selecionando valores únicos
df_UF = df_UF.dropna() # Dropando NA
df_UF.show # Visualizando tabela

DataFrame[CO_UF_ESC: int, SG_UF_ESC: string]

Salvando dados em tabela SQL

In [80]:
df_UF = df_UF.withColumnRenamed('CO_UF_ESC', 'cod_uf')\
        .withColumnRenamed('SG_UF_ESC', 'sigla_uf')

In [81]:
df_UF.write.format("jdbc").mode("append") .option("url", url) \
  .option("dbtable", "enem2020.uf") \
  .option("user", user) \
  .option("password", password) \
  .option("driver", "org.postgresql.Driver") \
  .save()

---
## Escolas

In [9]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import max

In [10]:
temp_df_escolaID = df.select('CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC').dropna().distinct()
df_temp_escola_ID = temp_df_escolaID.withColumn("ID_ESCOLA", monotonically_increasing_id())
df_temp_escola_ID.show()

+----------------+---------+----------------------+------------------+---------------+---------+
|CO_MUNICIPIO_ESC|CO_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|TP_SIT_FUNC_ESC|ID_ESCOLA|
+----------------+---------+----------------------+------------------+---------------+---------+
|         3138203|       31|                     2|                 1|              1|        0|
|         1505437|       15|                     2|                 1|              1|        1|
|         3522604|       35|                     2|                 1|              1|        2|
|         2933059|       29|                     2|                 1|              1|        3|
|         2914505|       29|                     2|                 1|              1|        4|
|         4300604|       43|                     2|                 1|              1|        5|
|         3300308|       33|                     4|                 1|              1|        6|
|         3202207|       32|  

In [60]:
df_temp_escola_ID.count()

10322

In [58]:
df.select('CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'NU_INSCRICAO').show()

+----------------+---------+----------------------+------------------+---------------+------------+
|CO_MUNICIPIO_ESC|CO_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|TP_SIT_FUNC_ESC|NU_INSCRICAO|
+----------------+---------+----------------------+------------------+---------------+------------+
|            null|     null|                  null|              null|           null|200006271946|
|            null|     null|                  null|              null|           null|200001195856|
|         2927408|       29|                     2|                 1|              1|200001943954|
|         3547304|       35|                     3|                 1|              1|200001908998|
|            null|     null|                  null|              null|           null|200001634757|
|            null|     null|                  null|              null|           null|200003132410|
|            null|     null|                  null|              null|           null|200001379770|


In [52]:
df.count()

5787745

In [11]:
df = df.join(df_temp_escola_ID, on=['CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC'], how = 'outer')

In [8]:
df_escolas = df.select(['ID_ESCOLA', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC']).dropna().distinct()

In [72]:
df_escolas.select(max('ID_ESCOLA')).show(2)

+--------------+
|max(ID_ESCOLA)|
+--------------+
|         10321|
+--------------+



In [9]:
df_escolas = df_escolas.withColumnRenamed('CO_MUNICIPIO_ESC', 'fk_cod_mun')\
                        .withColumnRenamed('CO_UF_ESC', 'fk_cod_uf')\
                        .withColumnRenamed('TP_DEPENDENCIA_ADM_ESC', 'dependencia_adm')\
                        .withColumnRenamed('TP_LOCALIZACAO_ESC', 'tipo_loc_escola')\
                        .withColumnRenamed('TP_SIT_FUNC_ESC', 'situacao_escola')

In [79]:
df_escolas.write.mode('append').format("jdbc").option("url", url) \
  .option("dbtable", "enem2020.escolas") \
  .option("user", user) \
  .option("password", password) \
  .option("driver", "org.postgresql.Driver") \
  .save()

### Tratamento de Erros - Caso participante não tenha registro de escola

Teremos o código 10322 para valores sem registros na base de dados oficial do ENEM.

In [12]:
df = df.fillna(subset=['ID_ESCOLA'], value = 10322)

<code>
insert into <br>
enem2020.escolas (<br><br>
    id_escola, <br>
    fk_cod_mun, <br>
    fk_cod_uf, <br>
    dependencia_adm, <br>
    tipo_loc_escola, <br><br>
    situacao_escola) values (10322, 9,9,9,9,9) </code>

---
## Municipio

In [11]:
df.printSchema()

root
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [12]:
df.select('CO_MUNICIPIO_PROVA').show(10)

+------------------+
|CO_MUNICIPIO_PROVA|
+------------------+
|           1501402|
|           3303401|
|           2408102|
|           2611606|
|           3121605|
|           3205200|
|           4305207|
|           4112108|
|           2611606|
|           2111300|
+------------------+
only showing top 10 rows



In [13]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat

In [28]:
df_mun_nome = df.select('NO_MUNICIPIO_PROVA').union(df.select('NO_MUNICIPIO_ESC').alias('NO_MUNICIPIO_PROVA'))
df_mun_cod = df.select('CO_MUNICIPIO_PROVA').union(df.select('CO_MUNICIPIO_ESC').alias('CO_MUNICIPIO_PROVA'))

In [29]:
df_mun_cod = df_mun_cod.withColumn("ID", monotonically_increasing_id())
df_mun_nome = df_mun_nome.withColumn("ID", monotonically_increasing_id())

In [30]:
df_mun = df_mun_cod.join(df_mun_nome,on = 'ID', how = 'inner').distinct().dropna()

In [35]:
df_mun = df_mun.select('CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA').distinct()
df_mun = df_mun.withColumnRenamed('CO_MUNICIPIO_PROVA', 'cod_mun')\
                .withColumnRenamed('NO_MUNICIPIO_PROVA', 'nome_municipio')

In [36]:
df_mun.write.mode('append').format("jdbc").option("url", url) \
  .option("dbtable", "enem2020.municipio") \
  .option("user", user) \
  .option("password", password) \
  .option("driver", "org.postgresql.Driver") \
  .save()

22/11/01 15:23:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


22/11/01 15:35:58 ERROR Executor: Exception in task 0.0 in stage 167.0 (TID 709)
java.sql.BatchUpdateException: Batch entry 30 INSERT INTO enem2020.municipio ("cod_mun","nome_municipio") VALUES (3131307,'Campina Grande') was aborted: ERROR: duplicate key value violates unique constraint "municipio_pkey"
  Detalhe: Key (cod_mun)=(3131307) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2367)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2099)
	at org.postgresql.core.v3.QueryExecutorImpl.flushIfDeadlockRisk(QueryExecutorImpl.java:1456)
	at org.postgresql.core.v3.QueryExecutorImpl.sendQuery(QueryExecutorImpl.java:1481)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:546)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(PgStateme

Py4JJavaError: An error occurred while calling o231.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 167.0 failed 1 times, most recent failure: Lost task 0.0 in stage 167.0 (TID 709) (192.168.0.138 executor driver): java.sql.BatchUpdateException: Batch entry 30 INSERT INTO enem2020.municipio ("cod_mun","nome_municipio") VALUES (3131307,'Campina Grande') was aborted: ERROR: duplicate key value violates unique constraint "municipio_pkey"
  Detalhe: Key (cod_mun)=(3131307) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2367)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2099)
	at org.postgresql.core.v3.QueryExecutorImpl.flushIfDeadlockRisk(QueryExecutorImpl.java:1456)
	at org.postgresql.core.v3.QueryExecutorImpl.sendQuery